In [12]:
#import tensorflow_datasets as tfds
import numpy as np
import re
import pandas as pd
import tensorflow as tf
import logging
# import tensorflow_text as text
# from transformers import BertTokenizer

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import os
import pathlib
root_folder='/content/drive/My Drive/WordGeneration'

In [42]:
import time
import math
import os
import gc

In [8]:
df = pd.read_csv(root_folder+'/FOMC2021.txt', sep="\n")
df.rename(columns={"Action to Adopt Changes to the Committee's Rules Regarding Availability of Information": "text"},
          inplace=True)
df["text"] = df["text"].str.replace("United States", "US")
df["text"] = df["text"].str.replace("U.S.", "US")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    486 non-null    object
dtypes: object(1)
memory usage: 3.9+ KB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [27]:
len_text = 40
len_result = 1
adj_lines = []
max_len = 3

In [25]:
BATCH_SIZE = 64

In [28]:
def text_cleaner(text):
    newString = text.lower()
    newString = re.sub("[\(\[].*?[\)\]]", "", newString)  # remove text in brackets
    newString = re.sub(r"'s\b", "", newString)
    newString = re.sub(r'[^\w\d\s\-]+', '', newString)
    newString = re.sub(r'[^\w\d\s]+', ' ', newString)
    newString = re.sub(r'"', '', newString)  # removes quotation marks
    newString = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", newString)  # all numbers
    # newString = re.sub(r'(?<=[^\s0-9])(?=[.,;:?])', r' ', newString)  # add space around punctuation
    newString = re.sub(r'\s\s', ' ', newString)  # removes double spaces
    return newString

In [29]:
lines = []
for t in df["text"]:
    lines.append(text_cleaner(t))

for i in range(len(lines)):
    if len(lines[i].split()) >= max_len:
        adj_lines.append(lines[i])

In [30]:
def make_sequences(texts,
                   training_length=len_text,
                   lower=True,
                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):

    # Create the tokenizer object and train on texts
    tokenizer = Tokenizer(lower=lower, filters=filters)
    tokenizer.fit_on_texts(texts)

    # Create look-up dictionaries and reverse look-ups
    word_idx = tokenizer.word_index
    idx_word = tokenizer.index_word
    num_words = len(word_idx) + 1
    word_counts = tokenizer.word_counts

    print(f'There are {num_words} unique words.')

    import pickle
    # Save the tokenizer
    with open('tokenizer.pickle', 'wb') as handle:
         pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    sequences = tokenizer.texts_to_sequences(texts)

    seq_lengths = [len(x) for x in sequences]
    over_idx = [
        i for i, l in enumerate(seq_lengths) if l > (training_length + len_result + 3)  # index and length of sequences
    ]

    new_texts = []
    new_sequences = []

    # Only keep sequences with more than training length tokens
    for i in over_idx:
        new_texts.append(texts[i])
        new_sequences.append(sequences[i])

    training_seq = []
    labels = []

    for seq in new_sequences:

        if len(training_seq) < 50000:
        # Create multiple training examples from each sequence
            for i in range(training_length, len(seq) - len_result):
                extract = seq[i - training_length:i + len_result]

                training_seq.append(extract[:len_text])
                labels.append(extract[len_text:])

    print(f'There are {len(training_seq)} training sequences.')

    return word_idx, idx_word, num_words, word_counts, new_texts, new_sequences, training_seq, labels

In [31]:
TRAINING_LENGTH = len_text
filters = '!"#$%&()*+/:<=>@[\\]^_`{|}~\t\n'
word_idx, idx_word, num_words, word_counts, new_texts, sequences, features, labels = \
    make_sequences(
    adj_lines, TRAINING_LENGTH, lower=True, filters=filters)

There are 2866 unique words.
There are 38883 training sequences.


In [32]:
def label_encode(labels, num_words):

    y_encoded = np.zeros((len(labels), num_words), dtype=np.int8)

    # One hot encoding of labels
    for example_index, word_index in enumerate(labels):
        y_encoded[example_index, word_index] = 1

    return y_encoded

In [33]:
encoded_labels = label_encode(labels, num_words)

In [34]:
# Define a dataset 
dataset = tf.data.Dataset.from_tensor_slices(
    (features, encoded_labels))
dataset = dataset.shuffle(len(np.array(df['text'])), reshuffle_each_iteration=True).batch(
    BATCH_SIZE, drop_remainder=True)

dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [53]:
features, labels = next(iter(dataset))

In [54]:
features

<tf.Tensor: shape=(64, 40), dtype=int32, numpy=
array([[  91,   38,   92, ...,   93,  110,   31],
       [   5,    1,  329, ...,  128,   26,   17],
       [   8,   57,   80, ...,  123,    1,  236],
       ...,
       [ 317,   23,   54, ...,   43,  158,  265],
       [  76,    8,    1, ...,    1,  137,   74],
       [1024,  924,    2, ...,  264,    4,   25]], dtype=int32)>

In [55]:
labels

<tf.Tensor: shape=(64, 2866), dtype=int8, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)>

In [ ]:
from transformers import BertTokenizer

print('Start building tokenizer ...')
tokenizer_en = BertTokenizer.from_pretrained("bert-base-cased")
print('End building tokenizer ...')

In [36]:
import sys
sys.path.append('/content/drive/My Drive/Summarisationv2')

In [37]:
import TransformerArchitecture
from TransformerArchitecture import *

In [40]:
# Set hyperparameters for the model
D_MODEL = 512 # 512
N_LAYERS = 4 # 6
FFN_UNITS = 512 # 2048
N_HEADS = 8 # 8
DROPOUT_RATE = 0.1 # 0.1

In [43]:
class EncoderTransformer(tf.keras.Model):
    
    def __init__(self,
                 vocab_size_enc,
                 d_model,
                 n_layers,
                 FFN_units,
                 n_heads,
                 dropout_rate,
                 name="EncoderTransformer"):
        super(EncoderTransformer, self).__init__(name=name)
        
        # Build the encoder
        self.encoder = Encoder(n_layers,
                               FFN_units,
                               n_heads,
                               dropout_rate,
                               vocab_size_enc,
                               d_model)

        # build the linear transformation and softmax function
        self.last_linear = layers.Dense(units=vocab_size_enc, name="lin_output")
    
    def call(self, enc_inputs, training):
        # Create the padding mask for the encoder
        enc_mask = create_padding_mask(enc_inputs)
        # Call the encoder
        enc_outputs = self.encoder(enc_inputs, enc_mask, training)
        # Call the Linear and Softmax functions
        outputs = self.last_linear(enc_outputs)
        
        return outputs

In [44]:
def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    loss_ = loss_object(target, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [49]:
def main_train(dataset, transformer, n_epochs, print_every=50):
  ''' Train the transformer model for n_epochs using the data generator dataset'''
  losses = []
  accuracies = []
  # In every epoch
  for epoch in range(n_epochs):
    print("Starting epoch {}".format(epoch+1))
    start = time.time()
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    # Get a batch of inputs and targets
    for (batch, (enc_inputs, labels)) in enumerate(dataset):

        with tf.GradientTape() as tape:
            # Call the transformer and get the predicted output
            predictions = transformer(enc_inputs, True)
            # Calculate the loss
            loss = loss_function(labels, predictions)
        # Update the weights and optimizer
        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
        # Save and store the metrics
        train_loss(loss)
        train_accuracy(labels, predictions)
        
        if batch % print_every == 0:
            losses.append(train_loss.result())
            accuracies.append(train_accuracy.result())
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch+1, batch, train_loss.result(), train_accuracy.result()))
            
    # Checkpoint the model on every epoch        
    ckpt_save_path = ckpt_manager.save()
    print("Saving checkpoint for epoch {} in {}".format(epoch+1,
                                                        ckpt_save_path))
    print("Time for 1 epoch: {} secs\n".format(time.time() - start))

  return losses, accuracies

In [50]:
# Clean the session
tf.keras.backend.clear_session()
# Create the Transformer model
transformer = EncoderTransformer(vocab_size_enc=num_words,
                          d_model=D_MODEL,
                          n_layers=N_LAYERS,
                          FFN_units=FFN_UNITS,
                          n_heads=N_HEADS,
                          dropout_rate=DROPOUT_RATE)

# Define a categorical cross entropy loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction="none")
# Define a metric to store the mean loss of every epoch
train_loss = tf.keras.metrics.Mean(name="train_loss")
# Define a matric to save the accuracy in every epoch
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
# Create the scheduler for learning rate decay
leaning_rate = CustomSchedule(D_MODEL)
# Create the Adam optimizer
optimizer = tf.keras.optimizers.Adam(leaning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)

In [51]:
#Create the Checkpoint 
ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, root_folder, max_to_keep=2)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Last checkpoint restored.")

In [ ]:
# Train the model
losses, accuracies = main_train(dataset, transformer, 5, 100)